<a id="TOC"></a> 
[save_dataframe_to_database](#save_dataframe_to_database)<br>
[integer_to_boolean](#integer_to_boolean)<br>
[df_column_count](#df_column_count)<br>
[pre_processing_nulls](#pre_processing_nulls)<br>
[pre_processing_dummies](#pre_processing_dummies)<br>
[pre_processing_scaling](#pre_processing_scaling)<br>

[TOC](#TOC)

In [1]:
import pandas as pd

import os
import sys

library_directory = os.path.abspath('C:\\Users\\jerry\\Programming\\library')
if library_directory not in sys.path:
    sys.path.append(library_directory)
from pre_processing_utilities import (
    pre_processing_nulls
    ,pre_processing_dummies
    ,pre_processing_scaling
    ,df_column_count
    ,integer_to_boolean
    ,scale_numeric_columns
)

from db_utils import (
    insert_csv_to_db
    ,csv_load_db
    ,save_dataframe_to_database
)
data = 'spotify'
database_name = 'spotify.db'
table_name = "spotify"
file_path = "spotify_songs.csv"
df = pd.read_csv(file_path)

C:\Users\jerry\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


   column_1  column_2 column_3
1       2.0      20.0        B
3       4.0      40.0        D


# save_dataframe_to_database
[TOC](#TOC)

In [2]:
database_name = 'spotify'
table_name = "spotify"
file_path = "spotify_songs.csv"
save_dataframe_to_database(database_name, table_name, df)

# integer_to_boolean
<a id="integer_to_boolean"></a> 
[TOC](#TOC)

In [3]:
numeric_column = 'track_popularity'
cutoff = 50
# Example usage
df_boolean = integer_to_boolean(df, numeric_column, cutoff=None, table_name=None, database=None)
df_boolean.head(1)

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,track_popularity_boolean
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,-2.634,1,0.0583,0.102,0.0,0.0653,0.518,122.036,194754,1


# df_column_count
[TOC](#TOC)

In [4]:
count_df = df_column_count(df, 'track_popularity_boolean')
count_df.head()

,track_popularity_boolean,count,percentage
0,1,16852,51.326409
1,0,15981,48.673591


# pre_processing_nulls

[TOC](#TOC)

In [1]:
import pandas as pd

import os
import sys

library_directory = os.path.abspath('C:\\Users\\jerry\\Programming\\library')
if library_directory not in sys.path:
    sys.path.append(library_directory)
from pre_processing_utilities import (pre_processing_nulls)

database_name = 'spotify'
table_name = "spotify"
file_path = "spotify_songs.csv"
df = pd.read_csv(file_path)

C:\Users\jerry\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


   column_1  column_2 column_3
1       2.0      20.0        B
3       4.0      40.0        D


In [8]:
import pandas as pd
import sqlite3

def query_database(database_path, query):
    """Queries the SQLite database and returns a dataframe."""
    with sqlite3.connect(database_path) as conn:
        result = pd.read_sql_query(query, conn)
    return result

def pre_processing_dummies(data):
    """Processes categorical variables by creating dummy variables.
    
    Accepts either a DataFrame or a SQLite table.
    
    Returns the processed DataFrame and non-numeric column names.
    """
    if isinstance(data, pd.DataFrame):
        df = data.copy()
    elif isinstance(data, str):  # Assuming string input is database path
        df = query_database(data, f"SELECT * FROM {data}")
    else:
        raise ValueError("Input must be either a DataFrame or a database path (string).")
    
    if df.empty:
        raise ValueError("DataFrame or table is empty.")
    
    non_numeric_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    df = pd.get_dummies(df, columns=non_numeric_cols, drop_first=True)
    
    return df, non_numeric_cols

database_path = 'spotify'
table_name = 'spotify'

processed_df, non_numeric_cols = pre_processing_dummies(database_path)

In [14]:
database_path = 'spotify.db'  # Adjust as per your database file location
table_name = 'spotify'
try:
    processed_df, non_numeric_cols = pre_processing_dummies(database_path, table_name)
    print("Processed DataFrame:")
    print(processed_df)
    print("Non-numeric columns:", non_numeric_cols)
except ValueError as e:
    print(str(e))

Processed DataFrame:
       track_popularity  danceability  energy  key  loudness  mode  \
0                    66         0.748   0.916    6    -2.634     1   
1                    67         0.726   0.815   11    -4.969     1   
2                    70         0.675   0.931    1    -3.432     0   
3                    60         0.718   0.930    7    -3.778     1   
4                    69         0.650   0.833    1    -4.672     1   
...                 ...           ...     ...  ...       ...   ...   
32828                42         0.428   0.922    2    -1.814     1   
32829                20         0.522   0.786    0    -4.462     1   
32830                14         0.529   0.821    6    -4.899     0   
32831                15         0.626   0.888    2    -3.361     1   
32832                27         0.603   0.884    5    -4.571     0   

       speechiness  acousticness  instrumentalness  liveness  ...  \
0           0.0583      0.102000          0.000000    0.0653  ...   


# pre_processing_scaling
<a id="pre_processing_scaling"></a> 
[TOC](#TOC)

In [19]:
import pandas as pd
data = 'spotify'
database_name = 'spotify.db'
table_name = "spotify"
file_path = "spotify_songs.csv"
df = pd.read_csv(file_path)

database_path = 'spotify'  # Adjust as per your database file location
table_name = 'spotify'  # Adjust table name as per your actual table

#(scale_numeric_columns(database_path, table_name)).head(3)

In [11]:
(scale_numeric_columns(df, table_name)).head(1)

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,0.941531,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,0.1732,1.367123,0.876177,-0.481362,-0.333898,-0.377953,-0.80923,0.031908,0.042927,-0.518874


In [1]:
import os
import sys

library_directory = os.path.abspath('C:\\Users\\jerry\\Programming\\library')
if library_directory not in sys.path:
    sys.path.append(library_directory)
from pre_processing_utilities import (
    pre_processing_nulls
    ,pre_processing_dummies
    ,pre_processing_scaling
)

C:\Users\jerry\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


   column_1  column_2 column_3
1       2.0      20.0        B
3       4.0      40.0        D


In [2]:
import pandas as pd
table_name = "spotify"
file_path = "spotify_songs.csv"
df = pd.read_csv(file_path)

In [ ]:
pre_processing_nulls
    ,pre_processing_dummies
    ,pre_processing_scaling

In [3]:
processed_df = pre_processing_nulls(df)
print("Processed DataFrame from SQLite table:")
print(processed_df)

Processed DataFrame from SQLite table:
                     track_id  \
0      6f807x0ima9a1j3VPbc7VN   
1      0r7CVbZTWZgbTCYdfa2P31   
2      1z1Hg7Vb0AhHDiEmnDE79l   
3      75FpbthrwQmzHlBJLuGdC7   
4      1e8PAfcKUYoKkxPhrHqw4x   
...                       ...   
32828  7bxnKAamR3snQ1VGLuVfC1   
32829  5Aevni09Em4575077nkWHz   
32830  7ImMqPP3Q1yfUHvsdn7wEo   
32831  2m69mhnfQ1Oq6lGtXuYhgX   
32832  29zWqhca3zt5NsckZqDf6c   

                                              track_name      track_artist  \
0      I Don't Care (with Justin Bieber) - Loud Luxur...        Ed Sheeran   
1                        Memories - Dillon Francis Remix          Maroon 5   
2                        All the Time - Don Diablo Remix      Zara Larsson   
3                      Call You Mine - Keanu Silva Remix  The Chainsmokers   
4                Someone You Loved - Future Humans Remix     Lewis Capaldi   
...                                                  ...               ...   
32828             